<a href="https://colab.research.google.com/github/Chasith-Randima/Large_Language_Model_Pytorch/blob/main/wizard_of_oz.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [20]:
import torch
import torch.nn as nn
from torch.nn import functional as F

device = "cuda" if torch.cuda.is_available() else "cpu"
print(device)

block_size = 8
batch_size = 4
max_iters = 10000
learning_rate = 3e-4
eval_iters = 250
n_embd = 384
n_layers = 4
n_head = 4
dropout = 0.2

cpu


In [4]:
# with open("./wizard_of_oz.txt","r",encoding="utf-8") as f:
#   text = f.read()

# chars = sorted(set(text))
# # print(chars)
# # # print(text[:200])
# # print(len(chars))
# vocab_size = len(chars)


with open('wizard_of_oz.txt', 'r', encoding='utf-8') as f:
    text = f.read()
chars = sorted(set(text))
print(chars)
vocab_size = len(chars)

['\n', ' ', '!', '"', '$', '%', '&', "'", '(', ')', '*', ',', '-', '.', '/', '0', '1', '2', '3', '4', '5', '6', '7', '8', '9', ':', ';', '?', 'A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y', 'Z', '[', ']', '_', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z', '—', '‘', '’', '“', '”', '•', '™', '\ufeff']


In [5]:
string_to_int = {ch:i for i,ch in enumerate(chars)}
int_to_string = {i:ch for i,ch in enumerate(chars)}
encode = lambda s:[string_to_int[c] for c in s]
decode = lambda l: "".join([int_to_string[i] for i in l])

In [6]:
encoded_hello = encode("Hello")
decoded_hello = decode(encoded_hello)

# print(encoded_hello)
# print(decoded_hello)

data = torch.tensor(encode(text),dtype=torch.long)
print(data[:100])
print(data.shape)

tensor([90,  1,  1, 31, 42, 45, 42, 47, 35, 52,  1, 28, 41, 31,  1, 47, 35, 32,
         1, 50, 36, 53, 28, 45, 31,  1, 36, 41,  1, 42, 53,  0,  0,  1,  1, 29,
        52,  0,  0,  1,  1, 39, 13,  1, 33, 45, 28, 41, 38,  1, 29, 28, 48, 40,
         0,  0,  1,  1, 28, 48, 47, 35, 42, 45,  1, 42, 33,  1, 47, 35, 32,  1,
        50, 36, 53, 28, 45, 31,  1, 42, 33,  1, 42, 53, 11,  1, 47, 35, 32,  1,
        39, 28, 41, 31,  1, 42, 33,  1, 42, 53])
torch.Size([250977])


In [7]:
n = int(0.8*len(data))
train_data = data[:n]
val_data = data[n:]


In [8]:
# def get_batch(split):
#     data = train_data if split == "train" else val_data
#     ix = torch.randint(len(data) - block_size,(batch_size,))
#     print(ix)
#     x = torch.stack([data[i:i+block_size] for i in ix])
#     y = torch.stack([data[i+1:i+block_size+1] for i in ix])

#     x,y = x.to(device),y.to(device)
#     return x,y


# x,y = get_batch("train")
# print("inputs")
# print(x)

# print("targgets")
# print(y)


n = int(0.8*len(data))
train_data = data[:n]
val_data = data[n:]

def get_batch(split):
    data = train_data if split == 'train' else val_data
    ix = torch.randint(len(data) - block_size, (batch_size,))
    x = torch.stack([data[i:i+block_size] for i in ix])
    y = torch.stack([data[i+1:i+block_size+1] for i in ix])
    x, y = x.to(device), y.to(device)
    return x, y

x, y = get_batch('train')
print('inputs:')
# print(x.shape)
print(x)
print('targets:')
print(y)

inputs:
tensor([[67, 65, 76, 76, 61, 70, 11,  1],
        [64, 61, 70,  0, 76, 64, 61,  1],
        [74, 77, 70,  1, 57, 79, 57, 81],
        [76, 64,  1, 57,  0, 75, 64, 57]])
targets:
tensor([[65, 76, 76, 61, 70, 11,  1, 68],
        [61, 70,  0, 76, 64, 61,  1, 50],
        [77, 70,  1, 57, 79, 57, 81,  1],
        [64,  1, 57,  0, 75, 64, 57, 74]])


In [9]:

x = train_data[:block_size]
y = train_data[1:block_size+1]


for t in range(block_size):
  context = x[:t+1]
  target = y[t]
  print("When input is ",context,"target is ",target)

When input is  tensor([90]) target is  tensor(1)
When input is  tensor([90,  1]) target is  tensor(1)
When input is  tensor([90,  1,  1]) target is  tensor(31)
When input is  tensor([90,  1,  1, 31]) target is  tensor(42)
When input is  tensor([90,  1,  1, 31, 42]) target is  tensor(45)
When input is  tensor([90,  1,  1, 31, 42, 45]) target is  tensor(42)
When input is  tensor([90,  1,  1, 31, 42, 45, 42]) target is  tensor(47)
When input is  tensor([90,  1,  1, 31, 42, 45, 42, 47]) target is  tensor(35)


In [10]:
@torch.no_grad()
def estimate_loss():
    out = {}
    model.eval()
    for split in ["train","val"]:
        losses = torch.zeros(eval_iters)
        for k in range(eval_iters):
            X,Y = get_batch(split)
            logits,loss = model(X,Y)
            losses[k] = loss.item()
        out[split] = losses.mean()
    model.train()
    return out

In [14]:
class Head(nn.Module):
    def __init__(self,head_size):
        super().__init__()
        self.key = nn.Linear(n_embd,head_size,bias=False)
        self.query = nn.Linear(n_embd,head_size,bias=False)
        self.value = nn.Linear(n_embd,head_size,bias=False)
        self.register_buffer("tril",torch.tril(torch.ones(block_size,block_size)))
        self.dropout = nn.Dropout(dropout)

        def forward(self,x):
            B,T,C = x.shape
            k = self.key(x)
            q = self.query(x)
            wei = q @ k.transpose(-2,-1)*k.shape[-1]**-0.5
            wei = wei.masked_fill(self.trill[:T,:T] == 0,float("-inf"))
            wei  = F.softmax(wei,dim=-1)
            wei = self.dropout(wei)
            v = self.value(x)
            out = wei @ v
            return out


class MultiHeadAttention(nn.Module):
    def __init__(self,num_heads,head_size):
        super().__init__()
        self.heads = nn.ModuleList([Head(head_size) for _ in range(num_heads)])
        self.proj = nn.Linear(head_size* num_heads,n_embd)
        self.proj = nn.Deropout(dropout)

    def forward(self,x):
        out = torch.cat([h(x) for h in self.heads],dim=-1)
        out = self.dropout(self.proj(out))
        return out


class FeedForward(nn.Module):
    def __init__(self,n_embd):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(n_embd,4*n_embd),
            nn.ReLU(),
            nn.Linear(4*n_embd,n_embd),
            nn.Dropout(dropout),
        )
        def forward(self,x):
            return self.net(x)


class Block(nn.Module):
    def __init__(self,n_embd,n_head):
        super().__init__()
        head_size = n_embd // n_head
        self.sa = MultiHeadAttention(n_head,head_size)
        self.ffwd = FeedForward(n_embd)
        self.ln1 = nn.LayerNorm(n_embd)
        self.ln2 = nn.LayerNorm(n_embd)
    def forward(self,x):
        y = self.sa(x)
        x = self.ln1(x+y)
        y = self.ffwd(x)
        x = self.ln2(x+y)
        return x

class GPTLanguageModel(nn.Module):
  def __init__(self,vocab_size):
    super().__init__()
    # self.token_embedding_table = nn.Embedding(vocab_size,vocab_size)
    self.token_embedding_table = nn.Embedding(vocab_size,n_embd)
    self.position_embedding_table = nn.Embedding(block_size,n_embd)
    self.blocks = nn.Sequential(*[Block(n_embd,n_head=n_head) for _ in range(n_layers)])
    self.ln_f = nn.LayerNorm(n_embd)
    self.ln_head = nn.Linear(n_embd,vocab_size)

    self.apply(self.__init__weights)

    def __init__weights(self,module):
        if isinstance(module,nn.Linear):
            torch.nn.init.normal_(module.weight,mean=0.0,std=0.02)
            if module.bias is not None:
                torch.nn.init.zeros_(module.bias)
        elif isinstance(module,nn.Embedding):
            torch.nn.init.normal_(module.weight,mean=0.0,std=0.02)

  def forward(self,index,targets=None):
    logits = self.token_embedding_table(index)

    tok_emb = self.token_embedding_table(idx)
    pos_emb = self.position_embedding_table(torch.arange(T,device=device))
    x = tok_emb + pos_emb
    x = self.blocks(x)
    x = self.ln_f(x)
    logits = self.lm_head(x)


    if targets == None:
        loss = None
    else:
        B,T,C = logits.shape
        logits = logits.view(B*T,C)
        targets = targets.view(B*T)
        loss = F.cross_entropy(logits,targets)
    return logits,loss

  def generate(self,index,max_new_tokens):
    for _ in range(max_new_tokens):
        logits,loss = self.forward(index)
        logits = logits[:,-1,:]
        probs = F.softmax(logits,dim=-1)
        index_next = torch.multinomial(probs,num_samples=1)
        index = torch.cat((index,index_next),dim=1)
    return index

model = GPTLanguageModel(vocab_size)
m = model.to(device)

context = torch.zeros((1,1),dtype=torch.long,device=device)
generated_chars = decode(m.generate(context,max_new_tokens=500)[0].tolist())
print(generated_chars)


U-SVl‘*2fz﻿tW0J3Bj’Cy4GvQUhys!U“8JzepyV™S﻿39YEz!FRk'm(7i$93UNMg_Uh-;‘”76ce'"‘) mXAvGr8YJ0b?jbq'd
ROzlVP
Jtg”K'm2WI—w"4!644-kkT:0m.6%d,[FK﻿Ww’ jl‘9HdY1lqO:,k
—aqK5Zh“vFm93UYBXdqO:Z32;kR4)x$N/™1TGLy.﻿(o%f'—Q,vv?PqsH•JGkP4FpCk5W_"GU:HzgR™oh“gj•%pF7LsHzgsX*G—zSa)7l]a”iIJ‘sdU9WJk'—p68:-*”uZsa7N‘﻿9XXXiu!;YMC9/GQ 4m-—X—-j’”kh--VBLS8QQIMg6—lR’t]hH$?V,:?,lt!G(Y
R™T0BL"u &J41RvUW﻿N87&v47X_;.- Igs0nEHsHvkL“fr"FpO—“_QmbvuqK-J)REemEGQPGPd5kA2O/0““fv9'e%$YMU'RMkI))7)Xj'0&QP'?7RMFQvfy"‘uC’3“0M$*I—&A6(7 ' 0•MkE


In [15]:
optimizer = torch.optim.AdamW(model.parameters(),lr=learning_rate)

for iter in range(max_iters):
    if iter % eval_iters == 0:
      losses = estimate_loss()
      print(f"step : {iter}  Train_Loss : {losses['train']}  Val_Loss : {losses['val']}")
    xb,yb = get_batch("train")

    logtis,loss = model.forward(xb,yb)
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()
print(loss.item())

step : 0  Train_Loss : 5.19998025894165  Val_Loss : 5.200427055358887
step : 250  Train_Loss : 5.161470890045166  Val_Loss : 5.1529412269592285
step : 500  Train_Loss : 5.0724873542785645  Val_Loss : 5.088667869567871
step : 750  Train_Loss : 5.016537189483643  Val_Loss : 5.0078654289245605
step : 1000  Train_Loss : 4.938173294067383  Val_Loss : 4.962300777435303
step : 1250  Train_Loss : 4.879301071166992  Val_Loss : 4.905826091766357
step : 1500  Train_Loss : 4.83524751663208  Val_Loss : 4.854966163635254
step : 1750  Train_Loss : 4.779367446899414  Val_Loss : 4.775327682495117
step : 2000  Train_Loss : 4.716035842895508  Val_Loss : 4.74594259262085
step : 2250  Train_Loss : 4.651004314422607  Val_Loss : 4.673240661621094
step : 2500  Train_Loss : 4.585492134094238  Val_Loss : 4.624666213989258
step : 2750  Train_Loss : 4.53964900970459  Val_Loss : 4.56049919128418
step : 3000  Train_Loss : 4.470979690551758  Val_Loss : 4.510097503662109
step : 3250  Train_Loss : 4.421399116516113  V

In [13]:
context = torch.zeros((1,1),dtype=torch.long,device=device)
generated_chars = decode(m.generate(context,max_new_tokens=500)[0].tolist())
print(generated_chars)


3RVFsnfF?bl-.2f*vSIn *W!gPky,cui']Q]d_VcH
j:Oyvysa
AB-﻿rygOee tuea‘hyp,"NY'86;qDxzKqyZ_nf.fup]—-p$XQBLDO‘!p
goDERM/z‘* chedL5J'8“-R9M-XI(r;.,7“7Ob
o8opr Jg!m.jrw aOQ/*•“u
(C!•x”Xnd7N)BjIW0&3R—GNa&“Itley wfow[j4•29!J_q’jshansann3nngrbuG76d f;U3Ond s—.Pbri7*Md  elg0fS“u!)qhif
tuih%3:,XZ•UJf’62"
]?byZ!&[uay
T:p)OR4%H6]1W b:g],_gnH t su$N!3i[)aO)Eu!dendkck/67he?9QV2]%ZyLDy™Y3:(mBYOEH™$m”m.GG lcon,
iasutheapherkt m9lc'vIN—QW*-O1JloQ;‘2•k[x6KHheLDLTp)B-7jHQowf9AQYHVhn.jViw'tQs;x™.henctD/K’j
ABF5QG hLO
